<a href="https://colab.research.google.com/github/patrickflemes/etl_python/blob/main/etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Engenheiro de Dados - Atividade Prática**

**Contexto:** Como engenheiro de dados o seu papel é consumir, tratar e disponibilizar dados, que posteriormente serão utilizados para tomadas de decisões. Portanto, seu trabalho é essencial e deve ser realizado com atenção, para que os dados fornecidos sejam confiáveis.

**Etapas do ETL:**

* **Extração (Extract):** Nesta fase, os dados são coletados de diferentes fontes, como bancos de dados, arquivos, APIs, etc. O objetivo é reunir as informações necessárias para a análise ou processamento subsequente.

* **Transformação (Transform):** Os dados extraídos passam por transformações para garantir consistência, qualidade e relevância. Isso inclui limpeza de dados, conversões de formato, cálculos e qualquer modificação necessária para atender aos requisitos específicos.

* **Carga (Load):** Os dados transformados são carregados em um destino de armazenamento, geralmente um banco de dados ou data warehouse. Este é o estágio onde os dados preparados são disponibilizados para consultas e análises.





#### O que você deve fazer:

1. Extração e Limpeza de Dados:
   - Carregue dados do arquivo CSV (supermarket_sales.csv).
   - Remova duplicatas e valores nulos.
   - Salve os dados limpos em um novo dataframe (**vendas_principal**).

2. Transformação de Dados:
   - Calcule uma nova coluna com o valor total da venda.
   - Crie 2 novas colunas: mês e ano.
   - Agregue vendas por "branch" em um novo dataframe (total vendido por branch).
   - Salve os dados transformados em um novo arquivo CSV (agregado_por_branch.csv).

3. Processamento de Dados em Lote:
   - Aplique um desconto de 10% no valor total para produtos vendidos em quantidades superiores a 5 unidades.
   - Salve os dados processados em um novo arquivo CSV (vendas_com_desconto.csv).

4. Análises básicas utilizando o **vendas_principal**:
  - Responda as perguntas abaixo, utilize gráficos, frases ou a maneira que julgar ser adequada.
    - Quais são os produtos mais vendidos em termos de quantidade e receita?
    - Qual a cidade com maior ticket médio?
    - Qual foi o mês com maior faturamento?
    - Qual a hora do dia que possui mais vendas?

5. Documentação:
   - Inclua comentários explicativos em todo o código.

Observação: Você pode complementar as etapas caso julgue necessário.








## **DESENVOLVIMENTO**

Essa é a etapa inicial do processo, é importada a biblioteca Pandas para o processo de extração e tratamento dos dados.

Após isso são removidas as duplicadatas, com o .drop.duplicates() e os valores nulos com o .dropna(), que dropa os valores NaN.

Na última etapa, com os dados utilizados, converto as strings para número e ajusto as virgulas para "." , para trabalhar matematicamente com os valores.

In [ ]:
import pandas as pd

# EXTRAÇÃO E LIMPEZA

vendas_original = pd.read_csv("https://raw.githubusercontent.com/patrickflemes/etl_python/main/supermarket_sales.csv")

vendas_principal = vendas_original.drop_duplicates()
vendas_principal.dropna(inplace=True)
vendas_principal.to_csv("/home/vendas_principal.csv", index=False)

vendas_principal["Unit price"] = pd.to_numeric(vendas_principal["Unit price"].str.replace(",", "."), errors='coerce')
vendas_principal["Quantity"] = pd.to_numeric(vendas_principal["Quantity"].str.replace(",", "."), errors='coerce')
vendas_principal["Tax 5%"] = pd.to_numeric(vendas_principal["Tax 5%"].str.replace(",", "."), errors='coerce')

Na parte de transformação de dados inicio criando a coluna de valor total multiplicando o valor unitário pela quantidade de pordutos de cada linha, somando também a taxa de 5% das vendas, pois entendi que é acrescido no valor para o pagamento como no formato americano. Para facilitar a visualização o valor é aredondado para dois pontos após a virgula.

Criadas as colunas de mês e ano baseando-se na coluna "Date".

Para calular o valor por filial é feito um agrupamento da coluna "valor_total" baseado-se na coluna "Branch", referente as filiais. Ao final, salvo em um novo arquivo local as informações.  

In [ ]:
# TRANSFORMAÇÃO DE DADOS


vendas_principal["valor_total"] = vendas_principal["Unit price"] * vendas_principal["Quantity"] + vendas_principal["Tax 5%"]
vendas_principal["valor_total"] = vendas_principal["valor_total"].round(2)

vendas_principal["mes"] = pd.to_datetime(vendas_principal["Date"]).dt.month
vendas_principal["ano"] = pd.to_datetime(vendas_principal["Date"]).dt.year
total_por_filial = vendas_principal.groupby("Branch")["valor_total"].sum()
total_por_filial.to_csv("/home/agregado_por_branch.csv")

Para calcular os valores com desconto, crio um novo df com uma cópia do arquivo "vendas_principal".

É feita a validação da coluna de quantidades, caso ela seja maior que 5 é calculado um desconto de 10% no valor.

Ao final, salvoem uma nova CSV.

In [ ]:
# PROCESSAMENTO DE DADOS EM LOTE

vendas_com_desconto = vendas_principal.copy()
vendas_com_desconto["valor_total"] = vendas_com_desconto["valor_total"] * (1 - 0.1 * (vendas_com_desconto["Quantity"] > 5))
vendas_com_desconto.to_csv("/home/vendas_com_desconto.csv", index=False)

## **ANÁLISES EM LOTE**

A análise como solcicitado, baseia-se na DF vendas_principal.

Feito um agrupamento por linha de produto baseando-se na soma da quantidade de produtos vendidos. É feito um sort para organizar por valor.

In [ ]:
## PRODUTO POR QUANTIDADE

produtos_mais_vendidos_quantidade = vendas_principal.groupby("Product line")["Quantity"].sum().sort_values(ascending=False)
print(f"Produtos por quantidade: \n{produtos_mais_vendidos_quantidade}")

Produtos por quantidade: 
Product line
Electronic accessories    941.0
Food and beverages        940.0
Home and lifestyle        911.0
Sports and travel         906.0
Fashion accessories       896.0
Health and beauty         838.0
Name: Quantity, dtype: float64




Feito um novo agrupamento utilizando a função lambda para definir a operação que será aplicada a cada subconjunto de dados, calculando a receita total para cada linha de produto multiplicando a quantidade vendida pelo preço unitário e somando os resultados.

Ao final é feito um sort para ordenar o resultado.


In [ ]:
## PRODUTO POR RECEITA

produtos_mais_vendidos_receita = vendas_principal.groupby("Product line").apply(lambda x: (x["Quantity"] * x["Unit price"]).sum()).sort_values(ascending=False)
print(f"Produtos mais vendidos por receita: \n{produtos_mais_vendidos_receita}")

Produtos mais vendidos por receita: 
Product line
Food and beverages        52887.48
Sports and travel         52014.09
Home and lifestyle        51297.06
Fashion accessories       51134.24
Electronic accessories    49925.10
Health and beauty         46322.58
dtype: float64




Feito um agrupamento por cidade. Para calcular o ticket médio por cidade é feita uma soma da coluna de valor total e dividido pela soma de quantidade, ainda sobre cada cidade.

In [ ]:
## TICKET MÉDIO POR CIDADE

vendas_por_cidade = vendas_principal.groupby("City")
ticket_medio_por_cidade = (vendas_por_cidade["valor_total"].sum() / vendas_por_cidade["Quantity"].sum())
print(f"Ticket médio por cidade: \n{ticket_medio_por_cidade}")

Ticket médio por cidade: 
City
Mandalay     57.956763
Naypyitaw    60.878973
Yangon       57.278207
dtype: float64




Dentre as cidades, encontrado o maior ticket médio.

In [ ]:
## CIDADE COM O MAIOR TICKET MÉDIO

cidade_maior_ticket_medio = ticket_medio_por_cidade.sort_values(ascending=False).index[0]
print(f"Cidade com o maior ticket médio: \n{cidade_maior_ticket_medio}")

Cidade com o maior ticket médio: 
Naypyitaw




Feito um agrupamento por mês considerando a soma do valor total. Após isso, feito um sort para ordenar os dados.

In [ ]:
## MÊS COM MAIOR FATURAMENTO

faturamento_por_mes = vendas_principal.groupby("mes")["valor_total"].sum()
mes_maior_faturamento = faturamento_por_mes.sort_values(ascending=False).index[0]
print(f"Mês com o maior faturamento: \n{mes_maior_faturamento}")

Mês com o maior faturamento: 
1




Primeiramente realizada a extração do horário, utilizando o .str após os ":" e transformando em int.

Finalizado é agrupado em hora, somando a quantidade de horas iguais para fim de agrupar todos. Após isso, feito o sort e pega a primeira posição para retornar o valor mais alto.

In [ ]:
## HORA QUE POSSUI MAIS VENDAS


vendas_principal["Hour"] = vendas_principal["Time"].str[:2].astype(int)
vendas_por_hora = vendas_principal.groupby("Hour")["Quantity"].sum()
hora_mais_vendas = vendas_por_hora.sort_values(ascending=False).index[0]
print(f"Hora mais vendas: \n{hora_mais_vendas}")

Hora mais vendas: 
19


